In [5]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
from client import *

## Set-up credential flow

In [6]:
def get_token(scope=None): 
    redirect_uri = "http://localhost:4000"
    token = util.prompt_for_user_token(user, scope, cid, secret, redirect_uri)
    return token

In [7]:
user=userID
cid=clientID
secret=secret

In [8]:
token = get_token(scope='playlist-read-private') # set the scope
sp = spotipy.Spotify(auth=token)

## Get list of all playlists I follow

In [14]:
def get_playlist_data():    
    playlists = sp.user_playlists(user)
    playlist_data = []
    
    while playlists:
        for i, playlist in enumerate(playlists['items']):
            playlist_data.append(playlist)
        if playlists['next']:
            playlists = sp.next(playlists)
        else:
            playlists = None
    
    return playlist_data

In [15]:
def get_playlist_uris(playlist_data):
    playlist_uris = []
    for playlist in playlist_data:
        playlist_uris.append(playlist['uri'])
    return playlist_uris

In [16]:
def get_playlist_jsons(playlist_uris):
    playlist_jsons = []
    for uri in playlist_uris:
        results = sp.playlist(uri)
        playlist_jsons.append(results)
    return playlist_jsons

In [17]:
def get_user_ids(playlist_jsons):
    userids = []
    for json in playlist_jsons:
        if json['owner']['id'] not in userids:
            userids.append(json['owner']['id'])
    return userids

In [18]:
playlist_data = get_playlist_data()

In [19]:
playlist_uris = get_playlist_uris(playlist_data)

In [20]:
playlist_jsons = get_playlist_jsons(playlist_uris)

In [21]:
userids = get_user_ids(playlist_jsons)

## Get playlist track info:

In [23]:
def get_track_info(playlist_jsons):
    playlist_dfs = []
    counter = 1
    for json in playlist_jsons:
        df = pd.DataFrame.from_dict(json['tracks']['items'])
        df['PID'] = counter
        counter += 1
        playlist_dfs.append(df)
    df = pd.concat(playlist_dfs, ignore_index=True)
    return df

In [26]:
df = get_track_info(playlist_jsons)

In [27]:
columns = ['added_at', 'track_name', 'track_id', 'artists', 'added_by', 'explicit', 'duration',  'album_name', 'album_id', 'release_date', 'PID']

In [28]:
# expand track data into cols
df['track_id'] = df['track'].apply(lambda x: x['id'] if x is not None else None,convert_dtype=False)
df['track_name'] = df['track'].apply(lambda x: x['name'] if x is not None else None,convert_dtype=False)
df['album_id'] = df['track'].apply(lambda x: x['album']['id'] if x is not None else None,convert_dtype=False)
df['album_name'] = df['track'].apply(lambda x: x['album']['name'] if x is not None else None,convert_dtype=False)
df['artists'] = df['track'].apply(lambda x: x['artists'] if x is not None else None,convert_dtype=False)
df['added_by'] = df['added_by'].apply(lambda x: x['uri'] if x is not None else None,convert_dtype=False)
df['explicit'] = df['track'].apply(lambda x: x['explicit'] if x is not None else None,convert_dtype=False)
df['duration'] = df['track'].apply(lambda x: (x['duration_ms'] / 1000) if x is not None else None,convert_dtype=False)
df['release_date'] = df['track'].apply(lambda x: x['album']['release_date'] if x is not None else None,convert_dtype=False)

In [29]:
df = df[columns]
df = df.dropna()

In [30]:
# maybe remove a['id']
def artist_to_array(artists):
    artists_new = []
    for a in artists:
        artists_new.append([a['name'], a['id']])
    return artists_new

In [31]:
df['artists'] = df['artists'].apply(artist_to_array)

In [32]:
# set correct types of columns
df = df.astype({'release_date': 'datetime64[ns]', 'added_at': 'datetime64[ns]', 'duration': 'float'})
df['age'] = (df['added_at'] - df['release_date']).dt.days

In [33]:
# rename columns to make names a bit _nicer_
column_name_dict = {'added_at':'date_added'}

def nice_column_names(x):
    x = x.replace('_', ' ')
    x = string.capwords(x)
    return x

df.rename(columns=column_name_dict, inplace=True)
# df.columns = list(map(nice_column_names, df.columns))

In [34]:
df.to_csv('./data/all-tracks-07-04.csv', index=False)

# Make playlist from track df

In [35]:
# run once, use playlist_id to ADD the songs to
def initialize_playlist():
    playlist = sp.user_playlist_create(user, 'sub2', public = False)
    playlist_id = playlist['id']
    
    return playlist_id

In [36]:
# add playlist metadata
def update_playlist_details(playlist_id):
    playlist_name = 'in a minute (or so)'
    image_b64 = myimage
    playlist_description = 'I wrote a piece of code to take every song from every playlist I follow or made to find all of the songs that are less than 2 minutes long. Spotipy. short songs, one minute songs, two minute songs, 30 second songs'
    sp.user_playlist_change_details(user, playlist_id, name=playlist_name, description=playlist_description)
#     sp.playlist_upload_cover_image(playlist_id=playlist_id, image_b64 = myimage)
    
    print("details updated")

In [37]:
def add_cover_image():
    sp.playlist_upload_cover_image(playlist_id=sub2_id, image_b64 = myimage)
    print("image added")

In [38]:
import base64
with open("./data/images/poodle.JPG", "rb") as img_file:
    myimage = base64.b64encode(img_file.read()).decode("utf-8")

In [39]:
token = get_token(scope='playlist-modify-private ugc-image-upload') # set the scope
# read in all user's playlist data
sp = spotipy.Spotify(auth=token)

In [40]:
sub2_id = initialize_playlist()
update_playlist_details(sub2_id)
add_cover_image()

details updated
image added


## add tracks

In [41]:
# split into batches:
def make_batches(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]

In [42]:
# in batches, add the songs to the playlist
def add_tracks_to_main_playlist():
    for batch in batches:
        sp.playlist_add_items(playlist_id=sub2_id, items=batch)
    return print("confirm tracks added in spotify app")

In [43]:
df_sub2 = df.loc[(df['duration'] < 119)]
df_sub2_track_id = df_sub2['track_id'].drop_duplicates()
sub2_tracks = df_sub2_track_id.tolist()
batches = list(make_batches(sub2_tracks, 99))

In [44]:
add_tracks_to_main_playlist()

confirm tracks added in spotify app
